In [28]:
import numpy as np
import pandas as pd
import wrangle_students
import wrangle
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

# Feature Engineering

For some definitions, handling outliers and missing values, scaling, and encoding may be considered feature engineering. Here we'll draw a distinction between data preparation, data preprocessing, and feature engineering.

- **data preparation**: the basic data cleaning necessary to get our data ready for exploration/analysis, e.g. correcting data types, fixing typos
- **data preprocessing**: further data transformation done for the sake of modeling, as oppsoed to exploration/analysis, e.g. scaling, imputing, encoding
- **feature engineering**: adding, combining, or removing features; usually with the help of domain knowledge

Feature engineering can happen as part of data exploration or modeling, and engineered featured are also commonly explored.

Some examples of feature engineering by this definition:

- domain-based conversion (example: farenheit to celsius, BMI calculation, log transformation)
- domain based cutoffs (example: age >= 18 = is_adult; also dates)

- add / subtract (example: zillow dataset: beds + baths = room_count; total_sqft - 200 * bedrooms - 40 * bathrooms = living_area)
- combine as booleans as a count (example: telco_churn: streaming + backups + ...  = service_count)
- multiply / divide (example: tips dataset: total_bill / size = price_per_person)
- ratios (example: tips dataset: tip / total_bill = tip percentage)

Simplify!

- categorical with many unique values to top 3 + "Other"
- categorical to boolean: pool count -> has pool
- continous -> categorical via binning (aka quantization or discretization) (example: income -> high, medium, low earner)

In this lesson we'll cover some *automated* **feature selection** methods, that is, methods for determining which features are the most important.

- SelectKBest
- Recursive Feature Elimination
- Sequential Feature Selection

In [2]:
# mycols = ['col1', 'col2', 'col3', 'col4']
# 
# feature_space = []
# scores = []
# for col in mycols:
#     feature_space.append(col)
#     model_object.fit(X[feature_space], y)
#     scores.append(model_object.score(X[feature_space], y))
    
# my_scores = dict(zip(feature_space, scores))

## Setup

In [5]:
df = wrangle_students.wrangle_grades()

In [8]:
train, val, test = wrangle.split_data(df)

In [9]:
train.shape, val.shape, test.shape

((60, 5), (21, 5), (21, 5))

In [10]:
train.head()

,student_id,exam1,exam2,exam3,final_grade
60,61,70,65,78,77
59,60,73,70,75,76
21,22,70,65,78,77
2,3,85,83,87,87
51,52,70,75,78,72


In [11]:
X_train, y_train = train[['exam1', 'exam2', 'exam3']], train.final_grade
X_val, y_val = val[['exam1', 'exam2', 'exam3']], val.final_grade
X_test, y_test = test[['exam1', 'exam2', 'exam3']], test.final_grade

## Select K Best

- looks at each feature in isolation against the target based on correlation
- fastest of all approaches covered in this lesson
- doesn't consider feature interactions
- After fitting: `.scores_`, `.pvalues_`, `.get_support()`, and `.transform`

In [13]:
kbest = SelectKBest(f_regression, k=2)
_ = kbest.fit(X_train, y_train)

In [14]:
kbest.pvalues_

array([2.03679579e-46, 1.47195414e-25, 2.30229464e-30])

In [15]:
kbest.scores_

array([1961.95852406,  328.39519279,  507.39846869])

In [19]:
kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), 
                             index=X_train.columns)
kbest_results

,p,f
exam1,2.036796e-46,1961.958524
exam2,1.471954e-25,328.395193
exam3,2.302295e-30,507.398469


In [21]:
X_train.columns[kbest.get_support()]

Index(['exam1', 'exam3'], dtype='object')

In [27]:
X_train_transform = pd.DataFrame(kbest.transform(X_train),
                                 columns=X_train.columns[kbest.get_support()],
                                 index=X_train.index
                                )
X_train_transform.head()

,exam1,exam3
60,70,78
59,73,75
21,70,78
2,85,87
51,70,78


## RFE

- Recursive Feature Elimination
- Progressively eliminate features based on importance to the model
- Requires a model with either a `.coef_` or `.feature_importances_` property
- After fitting: `.ranking_`, `.get_support()`, and `.transform()`

In [29]:
model = LinearRegression()

In [30]:
rfe = RFE(model, n_features_to_select = 2)
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [33]:
pd.DataFrame({'rfe_ranking' : rfe.ranking_}, index=X_train.columns)

,rfe_ranking
exam1,1
exam2,2
exam3,1


In [32]:
rfe.get_support()

array([ True, False,  True])

In [35]:
X_train_transform = pd.DataFrame(rfe.transform(X_train), index=X_train.index, columns=X_train.columns[rfe.get_support()])
X_train_transform.head()

,exam1,exam3
60,70,78
59,73,75
21,70,78
2,85,87
51,70,78


## Sequential Feature Selector

- progressively adds features based on cross validated model performance
- forwards: start with 0, add the best additional feature until you have the desired number
- backwards: start with all features, remove the worst performing until you have the desired number
- After fitting: `.support_`, `.transform`

In [36]:
model = LinearRegression()

In [37]:
sfs = SequentialFeatureSelector(model, n_features_to_select=2)
sfs.fit(X_train, y_train)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=2)

In [38]:
X_train_transformed = pd.DataFrame(sfs.transform(X_train), index=X_train.index, columns=X_train.columns[sfs.support_])
X_train_transformed.head()

,exam1,exam3
60,70,78
59,73,75
21,70,78
2,85,87
51,70,78


## Conclusion

- Simpler models handle change + variability better
- Use RFE to narrow down your features and find the best ones, if your dataset is large (> 1GB; `df.info()`) use select k best instead
- Remember: feature engineering is much more than feature selection!